### 클래스 newsCrawler
### 크롤링하고 파일에 저장
* 이전: 크롤링함수 클래스화
* 0127: 한국어 불용어 처리
* 0128: 기사 요약문 제외처리

In [1]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions

class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        for i in range(100, 102):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(i)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
        except exceptions.ElementNotVisibleException as e:
            pass
        except Exception as e:
            pass
    
    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL=[]
            
            # 본문 링크
            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]
            #print(temp.a.text)         
            conURL = temp.a.get('href')
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')
            
            # 기사 제목, 내용 추출
            # 기사 요약 부분있으면 본문에서 제외하기
            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            
            contents = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            
            # 이미지 추출
            images=soup2.find_all('span', {'class','end_photo_org'})

            for i in range(len(images)):
                imagesURL.append(images[i].find("img")["src"])
                #print(imagesURL[i])
            
            # 기사 제목과 내용, 이미지URL 각 리스트에 저장
            self.titleList.append(title)
            self.contentsList.append(contents)
            self.imageList.append(imagesURL)
        
        # 파일 저장
        self.saveNewsToFile()
    
    # 파일에 저장(DB에 저장)
    def saveNewsToFile(self):
        f = open('result.txt', 'w', encoding='utf-8')
        for i in range(len(self.titleList)):
            f.write(self.titleList[i] + "\n")
            f.writelines(self.contentsList[i]+"\n")
            f.writelines(self.imageList[i])
            f.writelines("\n\n")
        f.close()

In [2]:
session = requests.Session()

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")

c=newsCrawler()
c.mainCrawl()

### SentenceTokenizer 클래스
### :텍스트 크롤링, 문장 단위 분리, 명사 추출

In [1]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

from konlpy.tag import Okt



class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean2.txt', encoding = 'utf-8')]
        #self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        #,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
        
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        print(article.text)
        sentences = self.kkma.sentences(article.text)
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)\
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns

### GraphicMatrix클래스
### :TF-IDF모델, 그래프 생성

In [2]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

### Rank클래스
### :TextRank알고리즘 구현

In [3]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

### TextRank클래스
### 문서3줄 요약, 키워드 10개 추출

In [4]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)

        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

In [5]:
#url = 'http://v.media.daum.net/v/20170611192209012?rcmd=r'
url='https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=018&aid=0004835586'
textrank = TextRank(url)
for row in textrank.summarize(3):
    print(row)
    print()

print('keywords :',textrank.keywords())

22일 코스피 시황(자료 = 신한HTS)

[이데일리 조용석 기자] 22일 코스피 지수가 하락 마감하며 3150선 아래로 떨어졌다. 장 내내 개인 순매수와 기관의 순매도가 팽팽하게 맞선 가운데 외국인의 수급에 따라 보합권에서 등락을 반복했다.마켓포인트에 따르면 이날 코스피 지수는 전 거래일 대비 0.64%(20.21포인트) 하락한 3140.63에 마감했다. 전일 종가기준 사상 최고치를 경신하는 등 최근 3거래일 연속 상승했던 코스피 지수는 다소 숨을 고르는 모양새다.개인이 1조6121억원 순매수, 기관이 1조3711억원을 순매도하면서 힘겨루기를 하는 동안 외국인 수급에 따라 지수가 움직였다. 1000억원대 순매도 시작한 외국인이 오전 11시께 순매도 규모를 200억원대로 줄이면서 코스피 지수는 상승 전환했으나 이후 다시 매도물량을 늘리자 함께 하락했다. 외국인은 결국 2659억원 순매도로 마감했다. 기관에서는 금융투자가 6697억원으로 가장 순매도 규모가 컸다.프로그램 매매는 차익과 비차익을 더해 7218억원 순매도였다. 원-달러 환율은 5.0원 상승한 1103.20에 거래를 마쳤다.시가총액 상위는 하락 우위였다. 대장주인 삼성전자(005930)가 1%대 하락한 것을 포함해 SK하이닉스(000660), LG화학(051910), 현대차(005380), 삼성바이오로직스(207940), 셀트리온(068270), 기아차(000270), 현대모비스(012330), LG전자(066570), 삼성물산(028260) 등도 모두 하락 마감했다. 반면 외국인 순매수금액 1,2위인 NAVER(035420)와 삼성SDI(006400)는 6%대의 높은 상승세를 보였다.업종별로도 하락이 다소 많았다. △운수장비 △철강 및 금속 △금융업 △보험 등이 2%대 하락했고 △증권 △전기가스업 △은행 △유통업 등도 1%대 하락했다. △ 반면 비금속 광물은 4%대, 섬유의복은 3% 대 올랐고 이외에도 △서비스업 △음식료품 △종이,목재 등도 상승 마감했다.종목별로는 수소 관련주인 두산퓨얼셀1우(33

In [28]:
#text = '''조 바이든 새 미국 행정부가 신종 코로나바이러스 감염증(코로나19) 대응 등 다른 주요 현안에 먼저 집중할 수밖에 없는 만큼 한국이 미국에 대북 정책을 우선하라고 해서는 안 된다는 미국 전문가 의견이 나왔다.마이클 그린 미국 전략국제문제연구소(CSIS) 선임 부소장은 22일 최종현학술원과 CSIS가 '바이든 시대와 한반도'를 주제로 개최한 화상 세미나에서 "기후변화, 팬데믹, 경제회복, 러시아, 국내 테러 등 바이든 대통령이 다뤄야 할 현안이 엄청나다는 점을 기억해야 한다"면서 이렇게 말했다.그린 부소장은 2001년 당시 김대중 대통령이 조지 W. 부시 대통령과 회담하며 북한과 정상회담을 우선하도록 밀어붙였다면서 "이때의 나쁜 첫인상 때문에 한미동맹과 미국의 대북 정책이 6개월에서 1년은 역행했다"고 주장했다.부시 행정부가 출범한 직후에 열린 당시 회담에서 대북 화해협력 정책을 강조한 김대중 대통령과 강경한 대북관의 부시 대통령은 현격한 시각차를 드러냈다.또 부시 대통령이 김 대통령을 "디스 맨"(this man)으로 불러 외교 결례 논란이 불거졌고, 불편한 한미관계를 드러낸 장면으로 평가됐다.그린 부소장은 "반면 오바마팀은 한미동맹에 대한 특별한 의제 없이 들어왔으나 이명박 대통령이 핵정상회의와 민주주의 등 오바마 대통령이 중요시하는 현안에서 한국이 파트너가 될 수 있다는 비전을 제시해 오바마 대통령의 관심을 사로잡았다"고 말했다.그는 "청와대가 북한과 정상회담이나 극적인 대북 정책을 우선하라고 하지 않기를 바란다. 바이든 행정부는 그럴 준비가 안 됐다"며 한국이 민주주의, 코로나19, 중국 문제 등에 대해 먼저 협력 의지를 밝힐 것을 조언했다.마크 리퍼트 전 주한미국대사는 도널드 트럼프 행정부에서 양국이 방위비 분담, 한미워킹그룹 등에 "과도한 에너지를 소비했다"고 지적하고서 바이든 시대에는 양국이 긴밀한 협력을 통해 동맹을 강화할 것을 주문했다.그는 "양국은 안보, 경제와 같은 전통적인 동맹 현안에 적극적으로 관여하면서 북한, 무역, 인도·태평양 전략 등에 대한 입장이 더 일치할 수 있도록 동맹을 현대화해야 한다"고 주문했다.한미 자유무역협정(FTA) 협상을 담당했던 웬디 커틀러 전 미국 무역대표부(USTR) 부대표는 "경제 분야에서 양국이 역내 도전을 해결하기 위해 협력할 기회가 많다"고 말했다.그는 양국이 중국의 도전에 대한 대응으로 수출통제와 투자제한 정책을 조율하고, 중국으로부터 경제보복을 당한 우방을 지지하며, 중국에 대한 공급망 의존도를 줄이는 데 협력해 나갈 것을 제안했다.리처드 아미티지 전 국무부 부장관은 북핵 문제에 대해 "어떤 문제는 그냥 관리할 수밖에 없다"며 "핵확산은 막아야 하지만 북한에서 핵은 이미 확산했고 끝난 일"이라고 말했다.그러면서 "북한이 한국, 일본, 미국 국민들을 해치지 않는 한 당분간은 이 문제를 관리하는 게 우리가 할 수 있는 가장 합리적인 선택"이라고 주장했다.'''
text ='''[이데일리 조용석 기자] 22일 코스피 지수가 하락 마감하며 3150선 아래로 떨어졌다. 장 내내 개인 순매수와 기관의 순매도가 팽팽하게 맞선 가운데 외국인의 수급에 따라 보합권에서 등락을 반복했다.
22일 코스피 시황(자료 = 신한HTS)
마켓포인트에 따르면 이날 코스피 지수는 전 거래일 대비 0.64%(20.21포인트) 하락한 3140.63에 마감했다. 전일 종가기준 사상 최고치를 경신하는 등 최근 3거래일 연속 상승했던 코스피 지수는 다소 숨을 고르는 모양새다.

개인이 1조6121억원 순매수, 기관이 1조3711억원을 순매도하면서 힘겨루기를 하는 동안 외국인 수급에 따라 지수가 움직였다. 1000억원대 순매도 시작한 외국인이 오전 11시께 순매도 규모를 200억원대로 줄이면서 코스피 지수는 상승 전환했으나 이후 다시 매도물량을 늘리자 함께 하락했다. 외국인은 결국 2659억원 순매도로 마감했다. 기관에서는 금융투자가 6697억원으로 가장 순매도 규모가 컸다.

프로그램 매매는 차익과 비차익을 더해 7218억원 순매도였다. 원-달러 환율은 5.0원 상승한 1103.20에 거래를 마쳤다.

시가총액 상위는 하락 우위였다. 대장주인 삼성전자(005930)가 1%대 하락한 것을 포함해 SK하이닉스(000660), LG화학(051910), 현대차(005380), 삼성바이오로직스(207940), 셀트리온(068270), 기아차(000270), 현대모비스(012330), LG전자(066570), 삼성물산(028260) 등도 모두 하락 마감했다. 반면 외국인 순매수금액 1,2위인 NAVER(035420)와 삼성SDI(006400)는 6%대의 높은 상승세를 보였다.

업종별로도 하락이 다소 많았다. △운수장비 △철강 및 금속 △금융업 △보험 등이 2%대 하락했고 △증권 △전기가스업 △은행 △유통업 등도 1%대 하락했다. △ 반면 비금속 광물은 4%대, 섬유의복은 3% 대 올랐고 이외에도 △서비스업 △음식료품 △종이,목재 등도 상승 마감했다.

종목별로는 수소 관련주인 두산퓨얼셀1우(33626K)가 가격 제한폭까지 올랐고 두산퓨얼셀2우B(33626L)도 23% 넘게 상승했다. 조 바이든 미국 대통령 취임 후 친환경 관련주에 대한 기대감이 반영된 것으로 보인다. 태경케미컬(006890)도 26% 넘게 상승 마감했다.

이날 거래량은 10억9424만2000주, 거래대금은 23조9627억3200만원으로 집게됐다. 상한가 1개 종목을 포함해 383개 종목이 올랐고 하한가 없이 476개 종목이 하락했다. 47개 종목은 보합권에서 거래를 마쳤다.

한편 21일(현지시간) 뉴욕증권거래소(NYSE)에서 다우존스 30 산업평균지수는 전장보다 0.04%(12.37포인트)하락한 3만1176.01에 마감했다. 반면 스탠더드앤드푸어스(S&P) 500 지수는 전장보다 0.03%, 기술주 중심의 나스닥 지수는 0.55% 오른 채 장을 마쳤다. S&P500·나스닥 지수는 전일에 이어 다시 종가 기준 최고가를 경신했다.'''
textrank = TextRank(text)
for row in textrank.summarize(3):
    print(row)
    print()

print('keywords :',textrank.keywords())

['일리 기자 코스 지수 하락 마감 아래', '내내 개인 매수 기관 매도 팽팽하 맞선 가운데 외국인 수급 보합 등락 반복', '코스 자료 신한 마켓 포인트 코스 지수 거래 대비 포인트 하락 마감', '종가 기준 사상 최고 경신 최근 거래 연속 상승 코스 지수 다소 모양새', '개인 매수 기관 매도 힘겨루기 동안 외국인 수급 지수', '매도 시작 외국인 오전 매도 규모 코스 지수 상승 전환 이후 매도물량 하락', '외국인 도로 마감', '기관 금융투자 가장 매도 규모', '프로그램 매매 차익 비차 매도', '달러 환율 상승 거래', '시가 상위 하락 우위', '대장 주인 삼성 전자 하락 포함 하이닉스 화학 현대 삼성 바이오 직스 트리 기아 현대 모비스 전자 삼성 물산 모두 하락 마감', '반면 외국인 매수 금액 위인 삼성 대의 상승세', '업종 별로 하락 다소', '운수 장비 철강 금속 금융업 보험 하락 증권 기가스 은행 유통 하락', '반면 비금속 광물 섬유 복은 이외 서비스업 식료품 종이 목재 상승 마감', '종목 별로 수소 관련 주인 가격 제한 상승', '바이든 미국 대통령 취임 환경 관련 대한 대감 반영', '태경 케미컬 상승 마감', '거래량 거래 금은 집게', '상한 종목 포함 종목 종목 하락', '종목 보합 거래', '현지 뉴욕 증권 거래소 다우 존스 산업 평균 지수 전장 포인트 하락 마감', '반면 스탠더드 앤드 지수 전장 기술 중심 나스닥 지수 오른', '나스닥 지수 전일 종가 기준 최고 경신']
22일 코스 피 시황( 자료 = 신한 HTS) 마켓 포인트에 따르면 이날 코스 피 지수는 전 거래일 대비 0.64% (20.21 포인트) 하락한 3140.63에 마감했다.

전 일 종가기준 사상 최고치를 경신하는 등 최근 3 거래일 연속 상승했던 코스 피 지수는 다소 숨을 고르는 모양새다.

1000억원 대 순매도 시작한 외국인이 오전 11시께 순매도 규모를 200억원대로 줄이면서 코스 피 지수는 상승 전환했으나 이후 다시 매도물량을 늘리자 함께 하락했

In [29]:
stopwords = [line.rstrip('\n') for line in open('stopwords_korean2.txt', encoding = 'utf-8')]
#with open('stopwords_korean.txt', encoding = 'utf-8') as f:
    #lines = f.readlines()
print(stopwords)
print(len(stopwords))

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']
98


In [7]:
#text = '''조 바이든 새 미국 행정부가 신종 코로나바이러스 감염증(코로나19) 대응 등 다른 주요 현안에 먼저 집중할 수밖에 없는 만큼 한국이 미국에 대북 정책을 우선하라고 해서는 안 된다는 미국 전문가 의견이 나왔다.마이클 그린 미국 전략국제문제연구소(CSIS) 선임 부소장은 22일 최종현학술원과 CSIS가 '바이든 시대와 한반도'를 주제로 개최한 화상 세미나에서 "기후변화, 팬데믹, 경제회복, 러시아, 국내 테러 등 바이든 대통령이 다뤄야 할 현안이 엄청나다는 점을 기억해야 한다"면서 이렇게 말했다.그린 부소장은 2001년 당시 김대중 대통령이 조지 W. 부시 대통령과 회담하며 북한과 정상회담을 우선하도록 밀어붙였다면서 "이때의 나쁜 첫인상 때문에 한미동맹과 미국의 대북 정책이 6개월에서 1년은 역행했다"고 주장했다.부시 행정부가 출범한 직후에 열린 당시 회담에서 대북 화해협력 정책을 강조한 김대중 대통령과 강경한 대북관의 부시 대통령은 현격한 시각차를 드러냈다.또 부시 대통령이 김 대통령을 "디스 맨"(this man)으로 불러 외교 결례 논란이 불거졌고, 불편한 한미관계를 드러낸 장면으로 평가됐다.그린 부소장은 "반면 오바마팀은 한미동맹에 대한 특별한 의제 없이 들어왔으나 이명박 대통령이 핵정상회의와 민주주의 등 오바마 대통령이 중요시하는 현안에서 한국이 파트너가 될 수 있다는 비전을 제시해 오바마 대통령의 관심을 사로잡았다"고 말했다.그는 "청와대가 북한과 정상회담이나 극적인 대북 정책을 우선하라고 하지 않기를 바란다. 바이든 행정부는 그럴 준비가 안 됐다"며 한국이 민주주의, 코로나19, 중국 문제 등에 대해 먼저 협력 의지를 밝힐 것을 조언했다.마크 리퍼트 전 주한미국대사는 도널드 트럼프 행정부에서 양국이 방위비 분담, 한미워킹그룹 등에 "과도한 에너지를 소비했다"고 지적하고서 바이든 시대에는 양국이 긴밀한 협력을 통해 동맹을 강화할 것을 주문했다.그는 "양국은 안보, 경제와 같은 전통적인 동맹 현안에 적극적으로 관여하면서 북한, 무역, 인도·태평양 전략 등에 대한 입장이 더 일치할 수 있도록 동맹을 현대화해야 한다"고 주문했다.한미 자유무역협정(FTA) 협상을 담당했던 웬디 커틀러 전 미국 무역대표부(USTR) 부대표는 "경제 분야에서 양국이 역내 도전을 해결하기 위해 협력할 기회가 많다"고 말했다.그는 양국이 중국의 도전에 대한 대응으로 수출통제와 투자제한 정책을 조율하고, 중국으로부터 경제보복을 당한 우방을 지지하며, 중국에 대한 공급망 의존도를 줄이는 데 협력해 나갈 것을 제안했다.리처드 아미티지 전 국무부 부장관은 북핵 문제에 대해 "어떤 문제는 그냥 관리할 수밖에 없다"며 "핵확산은 막아야 하지만 북한에서 핵은 이미 확산했고 끝난 일"이라고 말했다.그러면서 "북한이 한국, 일본, 미국 국민들을 해치지 않는 한 당분간은 이 문제를 관리하는 게 우리가 할 수 있는 가장 합리적인 선택"이라고 주장했다.'''
text ='''국민의힘 오세훈 서울시장 예비후보가 25일 오전 서울 여의도 국회 소통관에서 권역별 정책 공약을 발표하고 있다. 연합뉴스국민의힘 오세훈 서울시장 예비후보가 25일 토론 평가단 구성이 당원 중심으로 이뤄져 있어 평가 결과를 왜곡한다며 토론평가단 해체를 주장하고 나섰다.오세훈 후보는 "당초 4.7 재·보궐선거 중앙당 공천관리위원회는 토론평가단 운영을 위해 서울 각 당협별로 해당 선거구 거주자, 공직선거법상 '투표권이 있는 자', 적극적 자발적 참여 의사가 있는 당원 및 비당원을 대상으로 50명씩 추천을 받은 후 추첨을 통해 1000명의 토론평가단을 구성했다"며 "하지만 현행 평가단은 100% 당협위원장의 추천을 받아 구성되었음은 물론 당원 중심의 평가결과가 시민평가라는 이름으로 언론에 보도됨으로서 심각한 여론 왜곡을 조장하고 있다"고 주장했다.그는 "실제 이러한 평가 결과에 대해 일반 시민들은 물론 당원들은 공정하지 못하다며 이의를 제기하고 있다"고 했다.이어 "심지어 어떤 평가단원은 TV토론을 시청하지 않았음에도 자동응답 시스템에 응답한 경우도 있다고 하니, 이러한 모든 불공정 사례에 대해서도 명명백백히 밝혀야 할 것"고 강조했다.그러면서 오 후보 측은 세가지 요구사항을 제시했다.요구 내용은 ▲토론평가단 해체 ▲토론평가단의 당원 중심 구성을 밝히고 공개사과 ▲사무총장에 구두로 전한 시정요구의 논의 여부 등이다.장성혁 기자 jsh0529@imaeil.com▶ 네이버에서 매일신문 구독하기▶ 매일신문 네이버TV 바로가기▶ 나눔의 기적, 매일신문 이웃사랑ⓒ매일신문 - www.imaeil.com'''
textrank = TextRank(text)
print(textrank.summarize())

print('keywords :',textrank.keywords())

['국민 서울시장 예비 후보 오전 서울 여의도 국회 소통 역별 정책 공약 발표', '연합 뉴스 국민 서울시장 예비 후보 토론 평가 구성 당원 중심 평가 결과 왜곡 토론 평가 해체 주장', '후보 당초 보궐선거 중앙 공천 관리 위원회 토론 평가 운영 위해 서울 당협별 해당 선거구 거주 공직 선거 투표 적극 자발 참여 의사 당원 당원 대상 추천 추첨 통해 명의 토론 평가 구성 현행 평가 협위 원장 추천 구성 당원 중심 평가 결과 시민 평가 이름 언론 보도 여론 왜곡 조장 주장', '실제 평가 결과 대해 일반 시민 당원 이의 제기', '심지어 평가 단원 토론 시청 자동 응답 시스템 응답', '하니 모든 불공정 사례 대해 서도 명명 백백 강조', '후보 측은 요구사항 제시', '요구 내용 토론 평가 해체 토론 평가 당원 중심 구성 공개 사과 사무 총장 구두 시정 요구 논의 여부', '장성 기자 네이버 매일 신문 구독 매일신문 네이버 바로 가기 나눔 기적 매일신문 이웃 사랑 매일신문']
['연합 뉴스 국민의 힘 오 세 훈 서울시장 예비후보가 25일 토론 평가 단 구성이 당원 중심으로 이뤄 져 있어 평가 결과를 왜곡한다며 토론 평가 단 해체를 주장하고 나섰다.', '오 세 훈 후보는 " 당초 4.7 재· 보궐선거 중앙당 공천관리 위원회는 토론 평가 단 운영을 위해 서울 각 당협별로 해당 선거구 거주자, 공직선거 법상 \' 투표권이 있는 자\', 적극적 자발적 참여 의사가 있는 당원 및 비당원을 대상으로 50 명씩 추천을 받은 후 추첨을 통해 1000명의 토론 평가 단을 구성했다 "며 " 하지만 현행 평가 단은 100% 당 협위 원장의 추천을 받아 구성되었음은 물론 당원 중심의 평가 결과가 시민평가라는 이름으로 언론에 보도됨으로서 심각한 여론 왜곡을 조장하고 있다" 고 주장했다.', '요구 내용은 ▲ 토론 평가 단 해체 ▲ 토론 평가 단의 당원 중심 구성을 밝히고 공개 사과 ▲ 사무총장에 구두로 전한 시정요구의 논의 여부 등이다.']
keywords : ['평가', '당원